# Préparation des données

## Chargement des données

In [1]:
import pandas as pd
file_path = '../data/relatives_v2.xlsx'
sheet_name = 'data'
df = pd.read_excel(file_path, sheet_name=sheet_name)

In [2]:
df.head()

,text,psr,annotation
0,Simone Veil occupe désormais une place particu...,qui ont défiguré notre continent et avili l’hu...,det
1,une force qui a fait d’elle une actrice déterm...,qui a fait d’elle une actrice déterminante de ...,det
2,la Constitution permet au Président de la Répu...,"qui, de cette possibilité, font une nécessité",det
3,Les heures que nous vivons sont de celles-là.,que nous vivons,det
4,Je veux aujourd’hui vous parler du mandat que ...,que le peuple nous a donné,det


In [3]:
df.describe()

,text,psr,annotation
count,433,429,433
unique,430,418,20
top,"C’est cela, le sens de ce contrat social qui f...",qui viennent,det
freq,2,3,195


## Prétraitement des données :



### Normalisation:

cette partie permet de nettoyer et uniformiser des textes en remplaçant des caractères spéciaux par leurs équivalents standards.

In [4]:
import re

def replace_characters(match: re.Match) -> str:
    char = match.group(0)
    replacements = {'’': "'",'´': "'",'`': "'",'‘': "'",'«': '"','»': '"','“': '"','”': '"','–': '-','—': '-','…': ' ',u'\xa0': ' ',}
    return replacements[char]

def normalize_text(text: str) -> str:
    pattern = r'[’´`‘«»“”–—…]'
    return re.sub(pattern, replace_characters, text).strip()

In [5]:
# test
text = "un text avec des caractères spéciaux ’´`‘«»“”–—…"
print(normalize_text(text))

un text avec des caractères spéciaux ''''""""--


In [6]:
# apply to the dataframe
df = df.map(normalize_text)

TypeError: expected string or bytes-like object

### Tokenization:

In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

**RegexpTokenizer** : permet de diviser une chaîne de caractères en sous-chaînes en utilisant une expression régulière.

In [7]:
from nltk import RegexpTokenizer

def tokenizer(text):
  return RegexpTokenizer(r'''\w'|\w+|[^\w\s]''').tokenize(text)

In [8]:
# Test
text = "Bonjour tout le monde ! il s'agit d'un test."
tokenizer(text)

['Bonjour',
 'tout',
 'le',
 'monde',
 '!',
 'il',
 "s'",
 'agit',
 "d'",
 'un',
 'test',
 '.']

In [9]:
# Application de la fonction tokenizer sur les colonnes Phrase et PSR
df['texts'] = df['texts'].apply(tokenizer)
df['PSR'] = df['PSR'].apply(tokenizer)

In [10]:
df.head()

,texts,PSR,annotation
0,"[Simone, Veil, occupe, désormais, une, place, ...","[qui, ont, défiguré, notre, continent, et, avi...",det
1,"[une, force, qui, a, fait, d', elle, une, actr...","[qui, a, fait, d', elle, une, actrice, détermi...",det
2,"[Les, heures, que, nous, vivons, sont, de, cel...","[que, nous, vivons]",det
3,"[Je, veux, aujourd, ', hui, vous, parler, du, ...","[que, le, peuple, nous, a, donné]",det
4,"[des, institutions, que, je, veux, changer]","[que, je, veux, changer]",det


### Labilisation:

L'objectif de cette partie est de transformer les labels en valeurs numériques et aligner les labels avec le texte tokenisé.

In [11]:
def annotate(text, psr, tag):
    # Initialize the annotation list with 0s
    annotation = [0] * len(text)

    # Define the labels
    labels = {'O': 0, 'det': 1, 'appo': 2, 'ambiguë': 3}

    # Find the start index of the subsequence psr in text
    psr_len = len(psr)
    text_len = len(text)

    for i in range(text_len - psr_len + 1):
        if text[i:i + psr_len] == psr:
            annotation[i] = labels[tag]
            break

    return annotation

In [14]:
# test de la fonction annotate

psr = tokenizer("qui a touché tous les camps depuis tant de mois et parfois tant d'années")
tag = 'appo'

words = tokenizer("cette frénésie, qui a touché tous les camps depuis tant de mois et parfois tant d'années, est indigne de nous et des principes de la République.")
labels = annotate(words, psr,tag)

label_names = ['O', 'DET', 'APPO', 'AMBIGUE']


line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

# Affichage aligné des tokens et de leurs tags
print(line1)
print(line2)

cette frénésie , qui  a touché tous les camps depuis tant de mois et parfois tant d' années , est indigne de nous et des principes de la République . 
O     O        O APPO O O      O    O   O     O      O    O  O    O  O       O    O  O      O O   O       O  O    O  O   O         O  O  O          O 


In [15]:
# Appliquer la fonction annotate à chaque ligne du DataFrame pour créer la colonne psr_tags
df['psr_tags'] = df.apply(lambda row: annotate(row['texts'], row['PSR'], row['annotation']), axis=1)

In [17]:
# Supprimer les colonnes PSR et annotation
df.drop(columns=['PSR','annotation'], inplace=True)

# Renommer la colonne Phrase en tokens
df.rename(columns={'texts': 'tokens'}, inplace=True)

# reinstialiser l'index
df.reset_index(names='id' ,inplace=True)


In [18]:

df.head()

,id,tokens,psr_tags
0,0,"[Simone, Veil, occupe, désormais, une, place, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,"[une, force, qui, a, fait, d', elle, une, actr...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,2,"[Les, heures, que, nous, vivons, sont, de, cel...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
3,3,"[Je, veux, aujourd, ', hui, vous, parler, du, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]"
4,4,"[des, institutions, que, je, veux, changer]","[0, 0, 1, 0, 0, 0]"


## Création Dataset Hugging Face

### Découpage du jeu de données en train, validation et test

In [19]:
#split the dataset into train , validation and test
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

In [20]:
# Supprimer les colonnes d'index auto-générées si elles existent
train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True,inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [21]:
train_df.head()

,id,tokens,psr_tags
0,197,"[Mes, chers, collègues, ,, scellons, donc, auj...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,330,"[La, loi, du, 26, janvier, 2016, de, modernisa...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,321,"[Nous, avons, par, ailleurs, pris, différentes...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,430,"[Ces, travaux, débutent, aujourd, ', hui, sous...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,173,"[Ceux, qui, profitent, du, rouleau, compresseu...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


### Préparation des datasets

In [24]:
from datasets import Dataset, Features, Sequence, Value, ClassLabel

In [25]:
# Définir les Features pour chaque colonne du DataFrame
features = Features({
    'id': Value(dtype='int32', id=None),
    'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
    'psr_tags': Sequence(feature=ClassLabel(names=['O', 'DET', 'APPO', 'AMBIGUE'], id=None), length=-1, id=None)
})

In [26]:
# Convertir le DataFrame en Dataset Hugging Face avec les Features spécifiés
train_ds = Dataset.from_pandas(train_df, features=features)
val_ds = Dataset.from_pandas(val_df, features=features)
test_ds = Dataset.from_pandas(test_df, features=features)

In [27]:
train_ds

Dataset({
    features: ['id', 'tokens', 'psr_tags'],
    num_rows: 281
})

In [28]:
from datasets import DatasetDict

# Recréation du DatasetDict
dataset = DatasetDict({
    'train': train_ds,
    'validation': val_ds,
    'test': test_ds
})

In [29]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'psr_tags'],
        num_rows: 281
    })
    validation: Dataset({
        features: ['id', 'tokens', 'psr_tags'],
        num_rows: 71
    })
    test: Dataset({
        features: ['id', 'tokens', 'psr_tags'],
        num_rows: 88
    })
})

### Sauvegarde des datasets et publication sur Hugging Face

In [ ]:
dataset_name = 'relatives_psr_fr'

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# Enregistrer le DatasetDict dans le hub Hugging Face
dataset.push_to_hub(dataset_name)